In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [ ]:
pesticides=pd.read_csv('pesticides.csv')
pesticides.head()

In [ ]:
rainfall=pd.read_csv('rainfall.csv')
rainfall.head()

In [ ]:
temperature=pd.read_csv('temperature.csv')
temperature.head()

In [ ]:
yield_data=pd.read_csv('yield.csv')
yield_data.head()

In [ ]:
data=pd.merge(yield_data,pesticides, on=['Area','Year'],suffixes=('_yield','_pesticides'))
data.head()

In [ ]:
data=None #dont use this unless you want to reset data file

In [ ]:
data=pd.merge(data,rainfall, left_on=['Area','Year'],right_on=['Country','Year'])
data.drop(columns=['Country'],inplace=True)
data.head()

In [ ]:
data=pd.merge(data,temperature, left_on=['Area','Year'],right_on=['Country','Year'])
data.drop(columns=['Country'],inplace=True)
data.head()

In [ ]:
column_names=data.columns.tolist()
print(column_names)

In [ ]:
data=data.drop(columns=['Unnamed: 0_yield',	'Domain_yield',	'Element_yield',	'Unit_yield',	'Unnamed: 0_pesticides',	'Domain_pesticides'	,'Element_pesticides','Item_pesticides','Unit_pesticides',	'Unnamed: 0_x',		'Statistics_x'	,'ISO3_x',	'Unnamed: 0_y',	'Statistics_y'	,'ISO3_y'])

In [ ]:
data= data.dropna()
data.head()

In [ ]:
encoder=LabelEncoder()
data['Area']=encoder.fit_transform(data['Area'])
data['Item_yield'] = encoder.fit_transform(data['Item_yield'])

In [ ]:
unique_values=data['Item_yield'].unique()
print(unique_values)

In [ ]:
features=['Area','Year','Value_pesticides','Rainfall - (MM)','Temperature - (Celsius)']
X=data[features]
y_class=data['Item_yield']
y_regress=data['Value_yield']
scaler=MinMaxScaler()
X_scaled=scaler.fit_transform(X)

In [ ]:
X_train_c,X_test_c,y_train_c,y_test_c=train_test_split(X_scaled,y_class,test_size=0.2,random_state=42)
X_train_c,X_val_c,y_train_c,y_val_c=train_test_split(X_train_c,y_train_c,test_size=0.1,random_state=42)
X_train_r,X_test_r,y_train_r,y_test_r=train_test_split(X_scaled,y_regress,test_size=0.2,random_state=42)
X_train_r,X_val_r,y_train_r,y_val_r=train_test_split(X_train_r,y_train_r,test_size=0.1,random_state=42)

In [ ]:
print(y_train_c)
print(y_val_c)

In [ ]:
model_c=Sequential([
    Dense(128,activation='relu',input_shape=(X_train_c.shape[1],)),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(len(unique_values),activation='softmax')
])
opt_c=Adam(learning_rate=0.001)
model_c.compile(optimizer=opt_c,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history_c=model_c.fit(X_train_c,y_train_c,validation_data=(X_val_c,y_val_c),epochs=5,batch_size=32)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(history_c.history['loss'],label='Training Loss')
plt.plot(history_c.history['val_loss'],label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
test_loss_c,test_accuracy=model_c.evaluate(X_test_c,y_test_c)
print(f'Test Accuracy: {test_accuracy}')
print(f'Test Loss: {test_loss_c}')

In [ ]:
model_r=Sequential([
    Dense(128,activation='relu',input_shape=(X_train_r.shape[1],)),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(1,activation='linear')
])
opt_r=Adam(learning_rate=0.001)
model_r.compile(optimizer=opt_r,loss='mean_squared_error',metrics=['mean_squared_error'])
history_r=model_r.fit(X_train_r,y_train_r,validation_data=(X_val_r,y_val_r),epochs=5,batch_size=32)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(history_r.history['loss'],label='Training Loss')
plt.plot(history_r.history['val_loss'],label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
test_loss_r,test_mse=model_r.evaluate(X_test_r,y_test_r)
print(f'Test MSE: {test_mse}')